In [ ]:
import sys
sys.path.append("./..")
from config import config

import torch
import json, os
from glob import glob
from tqdm import tqdm
import librosa
import numpy as np

from transformers import AutoModelForAudioClassification

In [ ]:
device = torch.device(config['cude_device']) 

In [ ]:
#load model
model = AutoModelForAudioClassification.from_pretrained(config['emo_attributes'], trust_remote_code=True).to(device)

#get mean/std
mean = model.config.mean
std = model.config.std

In [ ]:
with open(config['json_path'], 'r') as fp:
    all_data = json.load(fp)

In [ ]:
audio = glob(os.path.join(config['podcast']['path'], "*.wav"))
audio.sort()
audio = {os.path.basename(x).split('.')[0]:x for x in audio}

In [ ]:
for i, pod in enumerate(tqdm(list(all_data.keys()))):
    
    wav, sr = librosa.load(audio[pod], sr=16000)

    for key, seg in all_data[pod].items():
        if 'Happy' in all_data[pod][key]:
            continue
        if (seg['end'] - seg['start']) > 100:
            continue
        if len(np.unique(wav[int(seg['start']*sr):int(seg['end']*sr)])) == 1:
            continue

        x = list(wav[int(seg['start']*sr):int(seg['end']*sr)])
        x = torch.tensor(x).unsqueeze(0).to(device)
        mask = torch.ones(1, len(x)).to(device)

        with torch.no_grad():
            emo_pred = model(x, mask).cpu().detach().numpy()[0]

            
        # pred = model()['logits'][0]
        all_data[pod][key]['Arousal'] = float(emo_pred[0])
        all_data[pod][key]['Dominance'] = float(emo_pred[1])
        all_data[pod][key]['Valence'] = float(emo_pred[2])

In [ ]:
with open(config['json_path'], 'w') as fp:
    json.dump(all_data, fp)